In [4]:
import influxdb_client
import pandas as pd
BUCKET = "bssr"
ORG = "BSSR"
TOKEN = "K1h0Bk9xIbuJiihHAywIXFE_IKnpO_81NWaiRFLCokXC5PSUMDPwwIJQmR7-bnBKfCejTS_gCy-A8YfmIAyeHQ=="
URL = "http://localhost:8086"


client = influxdb_client.InfluxDBClient(
   url=URL,
   token=TOKEN,
   org=ORG
)


query_api = client.query_api()

In [25]:
query1 = ' from(bucket: "bssr") \
  |> range(start: 1697082484, stop: 1697093284) \
  |> filter(fn: (r) => r["_measurement"] == "mqtt_consumer") \
  |> filter(fn: (r) => r["_field"] == "target_power") \
  |> aggregateWindow(every: 100ms, fn: mean, createEmpty: false) \
  |> yield(name: "mean")  \
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value") \
  |> keep(columns: ["_time", "target_power"])'
result1 = query_api.query_data_frame(query1)

query2 = ' from(bucket: "bssr") \
  |> range(start: 1697082484, stop: 1697093284) \
  |> filter(fn: (r) => r["_measurement"] == "mqtt_consumer") \
  |> filter(fn: (r) => r["_field"] == "car_speed") \
  |> aggregateWindow(every: 100ms, fn: mean, createEmpty: false) \
  |> yield(name: "mean")  \
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value") \
  |> keep(columns: ["_time", "car_speed"])'

result2 = query_api.query_data_frame(query2)

In [36]:
query3 = ' from(bucket: "bssr") \
  |> range(start: 1697082484, stop: 1697093284) \
  |> filter(fn: (r) => r["_field"] == "MCMB") \
  |> filter(fn: (r) => r["topic"] == "bus_metrics/voltage/MCMB") \
  |> aggregateWindow(every: 100ms, fn: mean, createEmpty: false) \
  |> yield(name: "mean")  \
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value") \
  |> keep(columns: ["_time", "MCMB"]) \
  '
result3 = query_api.query_data_frame(query3)

In [37]:
query4 = ' from(bucket: "bssr") \
  |> range(start: 1697082484, stop: 1697093284) \
  |> filter(fn: (r) => r["_field"] == "MCMB") \
  |> filter(fn: (r) => r["topic"] == "bus_metrics/current/MCMB") \
  |> aggregateWindow(every: 100ms, fn: mean, createEmpty: false) \
  |> yield(name: "mean")  \
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value") \
  |> keep(columns: ["_time", "MCMB"]) \
  '
result4 = query_api.query_data_frame(query4)

In [38]:
df1 = result1[1]
df2 = result2[1]
df4 = result4[1]
df3 = result3[1]

In [39]:
df4 = df4.rename(columns={"MCMB":"current"})
df3 = df3.rename(columns={"MCMB":"voltage"})

In [40]:
df3.head()

,result,table,_time,voltage
0,_result,0,2023-10-12 03:48:19+00:00,115.965263
1,_result,0,2023-10-12 03:48:19.300000+00:00,116.427849
2,_result,0,2023-10-12 03:48:19.500000+00:00,116.694305
3,_result,0,2023-10-12 03:48:19.800000+00:00,116.816216
4,_result,0,2023-10-12 03:48:20+00:00,116.810326


In [42]:
df = df1.merge(df2)

In [43]:
df = df.merge(df3)

In [44]:
df = df.merge(df4)

In [45]:
df

,result,table,_time,target_power,car_speed,voltage,current
0,_result,0,2023-10-12 03:48:19+00:00,0.0,101.5,115.965263,16.366140
1,_result,0,2023-10-12 03:48:19.300000+00:00,0.0,139.0,116.427849,12.728689
2,_result,0,2023-10-12 03:48:19.500000+00:00,0.0,235.0,116.694305,10.759596
3,_result,0,2023-10-12 03:48:19.800000+00:00,0.0,93.0,116.816216,10.311051
4,_result,0,2023-10-12 03:48:20+00:00,0.0,116.0,116.810326,9.951753
...,...,...,...,...,...,...,...
28275,_result,0,2023-10-12 06:48:02.900000+00:00,0.0,0.0,111.010590,0.101714
28276,_result,0,2023-10-12 06:48:03.200000+00:00,0.0,0.0,110.999931,0.102135
28277,_result,0,2023-10-12 06:48:03.400000+00:00,48.0,0.0,110.995728,0.101866
28278,_result,0,2023-10-12 06:48:03.700000+00:00,53.0,0.0,110.994179,0.109744


In [46]:
df.to_csv("kai.csv")